In [0]:
# naming file path
raw_data_path = "dbfs:/FileStore/tables/GHarchive"

In [0]:
#reading file path and creating dataframe
df = spark.read.format("json").load(raw_data_path)

In [0]:
from pyspark.sql.functions import col

# Explode out all top level nested columns
exploded_column_df = df.select(col("actor.*"), col("created_at"), col("id"), col("org.*"), col("payload.*"), col("public"), col("repo.*"), col("type"))

# Rename exploded columns to properly identify column origins
exploded_column_renamed_df = exploded_column_df.toDF("actor_avatar_url", "actor_display_login", "actor_gravatar_id", "actor_id", "actor_login", "actor_url", "event_created_at", "event_id", "org_avatar_url", "org_gravatar_id", "org_id", "org_login", "org_url", "payload_action", "payload_before","payload_comment","payload_commits","payload_description","payload_distinct_size", "payload_forkee", "payload_head","payload_issue", "payload_master_branch","payload_member", "payload_number", "payload_pages", "payload_pull_request", "payload_push_id","payload_pusher_type", "payload_ref", "payload_ref_type", "payload_release", "payload_review", "payload_size", "event_public", "repo_id", "repo_name", "repo_url", "event_type")

In [0]:
from pyspark.sql.functions import to_date

# Add date simplified date column for partitioning by day
bronze_layer_df = exploded_column_renamed_df.withColumn("partition_date", to_date(col("event_created_at")))

In [0]:
display(bronze_layer_df)

In [0]:
contname = 'team1-project2' #azure storage account container
storage_acct_name = '20230821desa'

In [0]:
# Partition by day
bronze_layer_df.repartition(15).write.partitionBy("partition_date").parquet(f"abfss://{contname}@{storage_acct_name}.dfs.core.windows.net/BronzeLayer")